<a href="https://colab.research.google.com/github/stkisengese/titanic-survial-prediction-ml/blob/main/notebook/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Titanic Machine Learining - Iteration 1: Naive Baseline

In [2]:
import pandas as pd
import numpy as np

Predict all passagers die(survival = 0)

In [3]:
# Load data
test = pd.read_csv('../data/test.csv')
train = pd.read_csv('../data/train.csv')

In [4]:
# write submission with all passengers predicted as died
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": 0
})

# Save the submission file
submission.to_csv('../data/gender_submission.csv', index=False)

In [5]:
# print(submission.head(10))
# print(submission.tail(10))

Iteration 2: 

In [9]:
# Exploratory Data analysis
print("Shape: ", train.shape)
train.info()
train.describe()
train.head()
train.isnull().sum()

Shape:  (891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64